In [39]:
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
from skimage import filters
from skimage import morphology
import math
from scipy import spatial
%matplotlib qt

In [ ]:
# Loading the raw t1 map for subject 1 and the resulting thickness map
t1 = nib.load('raw_t1_subject_02.nii').get_fdata()
t1map = nib.load('thickness_map_subject_01.nii').get_fdata()

plt.imshow(t1map[:,128,:])


In [ ]:
# Applying a filter to the image to make segmentation easier
filt_t1 = filters.gaussian(t1,sigma=1)
plt.imshow(filt_t1[:,128,:])

In [48]:
# Finding the white matter surface 

wm = filt_t1 > 75       # Isolating the white matter 

med_wm = filters.median(wm)     # Applying a median filter 

dilw = morphology.binary_dilation(med_wm)   # Dilating the white mater

edge_wm = dilw.astype(float) - med_wm  # Substracting the white matter from the dilation to get the surface

#plt.imshow(wm[:,128,:])
#plt.imshow(med_wm[:,128,:])
#plt.imshow(edge_wm[:,128,:])

In [49]:
# Finding the gray matter surface 

gm = (filt_t1 < 75) & (filt_t1 > 55)    # Isolating the gray matter 

gm = gm.astype(float)           # Converting gray matter into float values

med_gm = filters.median(gm)    # Applying a median filter 

dilg = morphology.binary_dilation(med_gm)   # Dilating the gray mater

edge_gm = dilg.astype(float) - med_gm    # Substracting the gray matter from the dilation to get the surface

#plt.imshow(gm[:,128,:])
#plt.imshow(med_gm[:,128,:])
#plt.imshow(edge_gm[:,128,:])

In [ ]:
# Getting the outer edge of the gray matter (pial surface)

dilw2 = morphology.binary_dilation(edge_wm)  # Dilating the white matter surface

fedge_gm = edge_gm.astype(float) - dilw2 # Substracting the white matter surface from the gray matter surface to get only the outer surface

fedge_gm2 = fedge_gm > 0  # Inverting the image to get rid of negative values

final = edge_wm + fedge_gm2 # Combining both surfaces

#plt.imshow(dilw2[:,128,:])
#plt.imshow(fedge_gm[:,128,:])
#plt.imshow(fedge_gm2[:,128,:])
#plt.imshow(final[:,128,:])

In [ ]:
# Getting arrays of non zero coordinates for white and gray matter edges
n0_wm = np.argwhere(edge_wm == 1)
n0_gm = np.argwhere(fedge_gm2 == 1)
n0_wm.shape

In [ ]:
# Finding the distance between white and gray matter boundaries and 
# assigning the distance value to each white matter boundary point
for x in range(n0_wm.shape[0]):
    dists = np.sqrt(np.sum((n0_wm[x,:] - n0_gm)**2, axis=1))
    mindists = np.min(dists)
    edge_wm[n0_wm[x,0],n0_wm[x,1],n0_wm[x,2]] = mindists 
    #print(x)

In [ ]:
plt.imshow(edge_wm[:,128,:])

In [ ]:
n0_gmfull = np.argwhere(gm == 1)
n0_gmfull.shape

In [ ]:
for x in range(n0_gmfull.shape[0]):
    point = n0_gmfull[x,:]
    #print(point)
    close = n0_wm[spatial.KDTree(n0_wm).query(point, distance_upper_bound=50)[1]]
    #print(close)
    gm[point[0],point[1],point[2]] = edge_wm[close[0],close[1],close[2]]
    #print(x)

In [ ]:
plt.imshow(gm[:,128,:])